# Prompt Engineering

Wykorzystuję:
- OpenAI API
- Groq API
- lokalny LLM poprzez LM Studio

## 1. Najważniejsze techniki tworzenia promptów.

### Zero-Shot Prompting:

Model dostaje tylko jedno zapytanie bez żadnych przykładów i musi samodzielnie wygenerować odpowiedź.



In [4]:
# Import potrzebnych bibliotek

from openai import OpenAI
from groq import Groq
import requests

In [5]:
# Pobranie kluczy API ze zmiennych środowiskowych

client_openai = OpenAI()
client_groq = Groq()

In [4]:
# Prosty prompt do dowolnej edycji
# Można użyć prompt = input("Zadaj pytanie: ") by móc wprowadzić dowolne zapytanie
# bez konieczności edycji kodu

prompt = "Czym jest sztuczna inteligencja?"

In [5]:
# OpenAI API

response_openai = client_openai.chat.completions.create(
    model="gpt-4o",   # należy wpisać nazwę wybranego modelu
    messages=[{"role": "user", "content": prompt}]
)

In [6]:
# Groq API

response_groq = client_groq.chat.completions.create(
    model="mixtral-8x7b-32768",  # należy wpisać nazwę wybranego modelu
    messages=[{"role": "user", "content": prompt}]
)

In [7]:
# Lokalny LLM - Lm Studio

lm_studio_url = "http://localhost:1234/v1/completions"
headers = {"Content-Type": "application/json"}
data = {
    "model": "llama-2-7b",      # należy wpisać nazwę wybranego modelu w LM Studio (lub local-model)
    "prompt": prompt,
    "max_tokens": 100           # ograniczenie do 100 tokenów
}

response_lms = requests.post(lm_studio_url, headers=headers, json=data).json()

Porównanie wyników:

In [10]:
print("="*50)
print("Odpowiedź OpenAI:\n")
print(response_openai.choices[0].message.content)
print("="*50)

print("Odpowiedź Groq:\n")
print(response_groq.choices[0].message.content)
print("="*50)

print("Odpowiedź LM Studio:\n")
print(response_lms.get("choices", [{}])[0].get("text", "No response"))
print("="*50)

Odpowiedź OpenAI:

Sztuczna inteligencja (SI) to dziedzina nauki i technologii zajmująca się tworzeniem systemów i programów komputerowych, które potrafią wykonywać zadania wymagające inteligencji ludzkiej. Zadania te mogą obejmować rozpoznawanie wzorców, podejmowanie decyzji, rozumienie języka naturalnego, uczenie się, planowanie czy rozwiązywanie problemów.

Istnieje wiele podejść do tworzenia SI, ale najpopularniejsze z nich to uczenie maszynowe, które polega na tworzeniu modeli matematycznych na podstawie danych, oraz sieci neuronowe, które naśladują sposób, w jaki funkcjonuje ludzki mózg.

Sztuczna inteligencja jest wykorzystywana w wielu dziedzinach, takich jak medycyna, motoryzacja, finansowość, przemysł czy rozrywka. Przykładami zastosowań SI są asystenci głosowi, systemy rekomendacji, autonomiczne pojazdy czy narzędzia do analizy danych.
Odpowiedź Groq:

Sztuczna inteligencja (   
Artificial Intelligence, AI) to dziedzina informatyki i inżynierii, której celem jest tworzenie m

### Few-Shot Prompting:

Model dostaje kilka przykładów poprawnych odpowiedzi, zanim przetworzy nowe zapytanie.

In [13]:
prompt_2 = """
Klasyfikuj podane algorytmy uczenia maszynowego jako nadzorowane (Supervised), nienadzorowane (Unsupervised) lub ze wzmocnieniem (Reinforcement Learning).

Przykłady:
1. Algorytm: Drzewa decyzyjne → Supervised
2. Algorytm: K-Means → Unsupervised
3. Algorytm: Q-Learning → Reinforcement Learning
4. Algorytm: Sieci neuronowe → 

Odpowiedź:
"""

# OpenAI API

response_openai = client_openai.chat.completions.create(
    model="gpt-4o",   # należy wpisać nazwę wybranego modelu
    messages=[{"role": "user", "content": prompt_2}]
)

print("Odpowiedź GPT-4o:\n ", response_openai.choices[0].message.content)

Odpowiedź GPT-4o:
  Supervised

Sieci neuronowe mogą być używane zarówno w kontekście uczenia nadzorowanego, jak i nienadzorowanego, ale najczęściej kojarzone są z uczeniem nadzorowanym, zwłaszcza w kontekście takich zastosowań jak klasyfikacja obrazów czy przetwarzanie języka naturalnego.


### Chain-of-Thought Prompting:

Model jest instruowany, aby rozwiązać problem krok po kroku, wyjaśniając każdy etap.

In [14]:
prompt_3 = """
Jesteś specjalistą Data Science, Machine Learning i AI z wieloletnim stażem.
Wyjaśnij krok po kroku, osobie zupełnie początkującej, czym są:
- Data Science
- Uczenie Maszynowe
- AI oraz GenAI
"""

# OpenAI API

response_openai = client_openai.chat.completions.create(
    model="gpt-4o",   # należy wpisać nazwę wybranego modelu
    messages=[{"role": "user", "content": prompt_3}]
)

print("Odpowiedź GPT-4o:\n ", response_openai.choices[0].message.content)

Odpowiedź GPT-4o:
  Z przyjemnością wytłumaczę te pojęcia! Chociaż są one ze sobą powiązane, każdy z tych terminów odnosi się do nieco innego obszaru technologii i analizy danych.

### Data Science (Nauka o Danych)
1. **Definicja**: Data Science to dziedzina, która zajmuje się pozyskiwaniem, analizowaniem i interpretowaniem dużych zbiorów danych w celu uzyskania użytecznych informacji i przewidywań.

2. **Procesy**:
   - **Zbieranie danych**: Zbieranie danych z różnych źródeł, takich jak bazy danych, pliki tekstowe, API czy strony internetowe.
   - **Czyszczenie danych**: Usuwanie zduplikowanych, błędnych lub niekompletnych danych, aby dane były spójne i użyteczne.
   - **Analiza i eksploracja danych**: Analizowanie danych przy użyciu różnych metod statystycznych i wizualizacyjnych w celu odkrycia wzorców.
   - **Modelowanie**: Tworzenie modeli statystycznych lub maszynowych w celu prognozowania przyszłych trendów lub wyników.
   - **Wizualizacja danych**: Prezentacja wyników analizy w

## 2. Clarification Prompting

In [23]:
# Zdefiniowanie funkcji promptu

def chat_completion(prompt, model="gpt-4", temperature=0):   # wyższa temperatura = bardziej kreatywna odpowiedź
    res = client_openai.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    print(res.choices[0].message.content)

prompt = """
Kontekst: jestem początkującym w dziedzinie Data Science.
Staram się uczyć przynajmniej 3-4 razy w tygodniu.
Zadanie: zaproponij mi plan nauki.
Nie wykonuj tego zadania! Zamiast tego zadaj mi pytania dotyczące
kontekstu, abyś po udzieleniu przeze mnie odpowiedzi mógł lepiej wykonać to zadanie.
"""
chat_completion(prompt)

1. Jakie są Twoje podstawowe umiejętności matematyczne i statystyczne? Czy masz doświadczenie w tych dziedzinach?
2. Czy masz doświadczenie w programowaniu? Jeśli tak, to w jakim języku?
3. Czy masz doświadczenie z jakimkolwiek konkretnym narzędziem lub technologią związaną z Data Science, takim jak Python, R, SQL, Hadoop, Spark itp.?
4. Ile czasu możesz poświęcić na naukę w ciągu tygodnia?
5. Czy masz jakieś konkretne cele lub obszary, na których chciałbyś się skupić podczas nauki Data Science?
6. Czy preferujesz naukę samodzielną, czy może chciałbyś skorzystać z kursów online lub uczestniczyć w szkoleniach?
7. Czy jesteś zainteresowany konkretnymi dziedzinami Data Science, takimi jak analiza danych, uczenie maszynowe, sztuczna inteligencja, wizualizacja danych itp.?
8. Czy jesteś bardziej zainteresowany teorią czy praktycznym zastosowaniem Data Science?


## 3. Persona-Based Prompting + Structured Prompting

Generator wiadomości:

In [15]:
# import potrzebnych bibliotek

from typing import List

In [16]:
# Zdefiniowanie funkcji zapytania

def ask_chatgpt(messages):
    response = client_openai.chat.completions.create(
        model = "gpt-3.5-turbo",    # próba dla modelu gpt-3.5-turbo
        messages=messages
    )
    return response.choices[0].message.content

In [17]:
# Przypisanie roli dla modelu

prompt_role = """
    Jesteś specjalistą Data Science, który prowadzi technicznego bloga. 
    Twoim zadaniem jest pisanie artykułów w oparciu o podane FAKTY. \
    Przestrzegaj następujących instrukcji: TON, DŁUGOŚĆ i STYL
    """

In [18]:
# Zdefiniowanie funkcji generatora

def news_generator(
        facts: List[str], tone: str, lenght_words: int, style: str
):
    facts = ", ".join(facts)
    prompt_4 = f"{prompt_role} \
        FAKTY: {facts} \
        TON: {tone} \
        DŁUGOŚĆ: {lenght_words} słów \
        STYL: {style}"
    return ask_chatgpt([{"role": "user", "content": prompt_4}]) 

In [20]:
print(news_generator(
    ["W zeszłym tygodniu wydano książkę związaną z Data Science",
     "Jej tytuł to Python w Data Science",
     "Wydawnictwo Helion",
    ], 
    "entuzjazm",
    300,
    "wpis na blogu"))

W zeszłym tygodniu świat Data Science został wzbogacony o nowe źródło wiedzy - książkę "Python w Data Science". Jest to pozycja, której nie może zabraknąć w bibliotece żadnego entuzjasty analizy danych. Wydana przez renomowane wydawnictwo Helion, książka ta dostarcza kompleksowego narzędzia do pracy z językiem Python w kontekście naukowej analizy danych.

Autorzy książki dokładnie omawiają najważniejsze biblioteki Pythona wykorzystywane w Data Science, takie jak pandas, numpy czy matplotlib, prezentując ich zastosowanie w praktyce. Krok po kroku pokazują, jak budować modele predykcyjne, analizować dane i prezentować wyniki w sposób zrozumiały dla wszystkich czytelników.

Jednym z atutów tej pozycji jest również podejście praktyczne autorów do tematu. Dzięki licznie przykładom i ćwiczeniom czytelnik ma możliwość nie tylko zdobyć teoretyczną wiedzę, ale także ją przetestować i zastosować w praktyce. To doskonały sposób na doskonalenie umiejętności w zakresie Data Science.

Książka "Pytho

In [21]:
print(news_generator(
    ["Pojawiła się aktualizacja biblioteki Pandas"
    ], 
    "techniczny",
    100,
    "news na blogu"))

W świecie Data Science pojawiła się właśnie aktualizacja biblioteki Pandas! Nowa wersja ma wiele nowych funkcji, które ułatwią analizę danych. Teraz użytkownicy mogą korzystać z jeszcze bardziej zaawansowanych narzędzi do manipulacji i wizualizacji danych. Dodatkowo, poprawiono wydajność biblioteki, co przyspieszy pracę z dużymi zbiorami danych. Dla wszystkich entuzjastów analizy danych to świetna wiadomość! Nie czekaj, już teraz zaktualizuj swoją bibliotekę Pandas i ciesz się jeszcze lepszymi możliwościami analizy danych. Ostatnie zmiany z pewnością ułatwią Ci pracę i przyspieszą proces tworzenia modeli danych.


## 4. Test DeepSeek vs GPT o1-mini

In [1]:
# Import wymaganych bibliotek

from IPython.core.display import display, HTML

C:\Users\rober\AppData\Local\Temp\ipykernel_30760\701484417.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
prompt_5 = "Czym jest generatywna AI"

In [7]:
# OpenAI API

response_openai_test = client_openai.chat.completions.create(
    model="o1-mini",   # należy wpisać nazwę wybranego modelu
    messages=[{"role": "user", "content": prompt_5}]
)

In [8]:
# Groq API

response_groq_test = client_groq.chat.completions.create(
    model="deepseek-r1-distill-llama-70b",  # należy wpisać nazwę wybranego modelu
    messages=[{"role": "user", "content": prompt_5}]
)

In [9]:
resp1 = response_openai_test.choices[0].message.content
resp2 = response_groq_test.choices[0].message.content

In [13]:
html_table = f"""
<table border="1" style="border-collapse: collapse; width: 100%;">
    <tr>
        <th style="padding: 10px; text-align: left;">Model OpenAI</th>
        <th style="padding: 10px; text-align: left;">Model Groq</th>
    </tr>
    <tr>
        <td style="padding: 10px; text-align: left; white-space: pre-wrap; font-family: Arial, sans-serif;">
            <pre style="white-space: pre-wrap; text-align: left;">{resp1}</pre>
        </td>
        <td style="padding: 10px; text-align: left; white-space: pre-wrap; font-family: Arial, sans-serif;">
            <pre style="white-space: pre-wrap; text-align: left;">{resp2}</pre>
        </td>
    </tr>
</table>
"""

display(HTML(html_table))

Model OpenAI,Model Groq
"Generatywna sztuczna inteligencja (ang. **Generative AI**) to gałąź sztucznej inteligencji, która koncentruje się na tworzeniu nowych treści, takich jak teksty, obrazy, muzyka, wideo czy nawet kod komputerowy, na podstawie dostarczonych danych treningowych. W przeciwieństwie do tradycyjnych systemów AI, które zazwyczaj analizują dane i podejmują decyzje na ich podstawie, generatywna AI potrafi tworzyć oryginalne i często zaskakujące rezultaty. ### Jak działa generatywna AI? Generatywna AI wykorzystuje różne modele uczenia maszynowego, z których najpopularniejsze to: 1. **Generative Adversarial Networks (GANs)** – Składają się z dwóch sieci neuronowych: generatora, który tworzy nowe dane, oraz dyskryminatora, który ocenia, czy dane są autentyczne (pochodzą z zestawu treningowego) czy wygenerowane. Obie sieci uczą się wspólnie, poprawiając swoje umiejętności w miarę treningu. 2. **Transformery** – Modele takie jak GPT (Generative Pre-trained Transformer) od OpenAI, które są trenowane na ogromnych zbiorach danych tekstowych i potrafią generować spójne i kontekstowo odpowiednie teksty na podstawie podanych wskazówek. 3. **Autoenkodery** – Modele, które uczą się kompresować i dekompresować dane, co pozwala na generowanie nowych przykładów podobnych do danych treningowych. ### Zastosowania generatywnej AI Generatywna AI ma szerokie zastosowania w różnych dziedzinach, takich jak: - **Tworzenie treści**: Pisanie artykułów, opowiadań, scenariuszy czy generowanie odpowiedzi na pytania. - **Sztuka i design**: Tworzenie grafik, ilustracji, projektów mody czy muzyki. - **Gry komputerowe**: Generowanie nowych poziomów, postaci czy narracji. - **Medycyna**: Projektowanie nowych leków poprzez generowanie struktur molekularnych. - **Inżynieria i architektura**: Projektowanie nowych produktów, budynków czy systemów. ### Przykłady narzędzi generatywnej AI - **ChatGPT**: Model językowy rozwinięty przez OpenAI, który potrafi prowadzić rozmowy, odpowiadać na pytania i generować teksty na różne tematy. - **DALL-E**: Model generatywny także od OpenAI, który potrafi tworzyć obrazy na podstawie opisów tekstowych. - **DeepArt**: Narzędzie przekształcające zdjęcia w styl różnych artystów. - **RunwayML**: Platforma oferująca narzędzia do tworzenia treści multimedialnych z wykorzystaniem AI. ### Wyzwania i etyka Chociaż generatywna AI oferuje wiele możliwości, niesie ze sobą także wyzwania: - **Prawa autorskie**: Tworzenie treści na podstawie istniejących dzieł może rodzić kwestie prawne. - **Dezinformacja**: Generowanie fałszywych informacji, zdjęć czy wideo może być wykorzystywane do manipulacji opinią publiczną. - **Bezpieczeństwo**: Modele mogą być wykorzystywane do tworzenia szkodliwych treści, takich jak deepfake. - **Etyka**: Decyzje dotyczące, jakie treści są generowane i jak są wykorzystywane, muszą być odpowiedzialne i zgodne z wartościami społecznymi. ### Podsumowanie Generatywna sztuczna inteligencja to potężne narzędzie, które rewolucjonizuje sposób tworzenia i interakcji z treściami cyfrowymi. Jej zdolność do generowania nowych, kreatywnych rozwiązań otwiera drzwi do innowacji w wielu dziedzinach, jednocześnie wymagając odpowiedzialnego podejścia do jej rozwoju i zastosowania.","Alright, the user is asking ""Czym jest generatywna AI"" which means ""What is generative AI"" in Polish. I should start by explaining that generative AI is a subset of artificial intelligence focused on creating content. It uses machine learning models, especially GANs and transformers, to generate text, images, videos, and more. I'll mention the training process, where models learn from large datasets to understand patterns and generate new content. Examples like DALL-E for images and GPT for text will help illustrate the concept. I should also highlight the applications in areas like art, writing, and even entertainment, showing how versatile generative AI is. Finally, I'll wrap it up by emphasizing the transformative potential of generativ